
# 

#  CuPy Lab 1
---

## Learning Objectives
- **The goal of this lab is to:**
     - enable you to quickly start using CuPy (beginner to intermediate level)
     - teach you to apply the concepts of GPU programming to HPC field(s); and
     - show you how to achieve a computational speedup on the GPU to maximize the throughput of your HPC implementation.

Before we begin, let's execute the cell below to display information about the CUDA driver and GPUs running on the server by running the `nvidia-smi` command. To do this, execute the cell block below by clicking on it with your mouse, and pressing Ctrl-Enter, or pressing the play button in the toolbar above. You should see some output returned below the grey cell.

In [ ]:
!nvidia-smi



##  Introduction
CuPy is an open-source library that implements GPU-accelerated NumPy arrays on CUDA. CuPy represents a GPU version of NumPy. NumPy runs only on CPU cores while CuPy leverages on multiple CUDA cores for parallel execution, therefore, CuPy is considered to run fastest and delivers maximum speed up. Due to the NumPy-compatibility nature of CuPy, almost all NumPy functionalities including multi-dimensional arrays and data types are implemented by CuPy. The rest of this notebook includes simple illustration on `CuPy architecture`, `CuPy fundamentals`, `CuPy CUDA kernels` and, frequently use terms like `Host` (this refers to a CPU), `Device` (means a GPU), and `Kernel` (a CuPy user-defined function that runs on the GPU).
   
   
## CuPy Architecture
The CuPy architecture exposes functionalities within the CuPy API that allows developers (or users) to create a user-defined CUDA kernel and make use of deep neural network utility through the `cuDNN` functionality. Linear algebras are solved through `cuBLAS` while systems of equations are solved with `cuSOLVER`. The `cuSPARSE` and `cuTENSOR` API functions specifically target sparse matrix and tensor operations respectively. Random numbers are generated using `cuRAND`. Sort, Scan and Reduction operations are conveniently executed using `CUB` and `Thrust`. Furthermore, Multi-GPU data transfer tasks are initiated with `NCCL` functionality. It is important to know that all these API functionalities rely on `CUDA`, while the CUDA itself depends on the `NVIDIA GPU` as shown in figure 1.0.
<center><img src="../images/cupy_arch.png" height="416px" width="506px"></center>
<center><div>Figure 1.0 CuPy Architecture</div></center> 

##  CuPy Fundamentals

In this section, three frequently used CuPy paradigm namely variable initialization, data transfer, and device selection would be considered. 

- **Variable or data initialization**: This is the process of assigning data or value to CuPy ndarray. The first step is to import the CuPy library and then initialize variables with data type as follows:
```python 
import cupy as cp
X1 = cp.array([1,2,3,4,5,6,7,8,9,10], dtype=cp.int32)#array of 10 values
X2 = cp.arange(100, dtype=cp.float32)#generating array of 100 values 
X3 = cp.empty((3,3), dtype=cp.float32)#initializing empty 2D array of 3X3 matrix
Sizebin = 10000
X4 = cp.zeros(sizebin, dtype=cp.int64)#initializing array filled with 10,000 zeros
```


- **Data transfer**: The idea is to move or copy data from the Host (NumPy) to the Device (CuPy) and vice versa such that data is visible to the Kernel and the resulting output would be copied back to the Host.
```python
import numpy as np
import cupy as cp
#copy data from Host to Device using cp.asarray()
h_X = np.arange(100, dtype=np.float32)#generating array of 100 values on the Host with NumPy
d_X = cp.asarray(x)# copy data to Device 
#copy data from Device to Host using cp.asnumpy()
h_X = cp.asnumpy(d_X)
```


- **Device selection**: This is a mechanism used by CuPy to select a particular GPU or switch from one Device to another (when there are more than one Device, default device is given 0 index id).
```python
Using default Device
X1 = cp.array([1,2,3,4,5,6,7,8,9,10], dtype=cp.int32)
```
Switching Devices
```python
cp.cuda.Device(1)
X1 = cp.array([1,2,3,4,5,6,7,8,9,10], dtype=cp.int32)
```


Switch GPU temporarily to GPU index 2 (minimum of 3 GPUs must exist to use index 2)
```python
with cp.cuda.Device(2): 
	X2 = cp.arange(100, dtype=cp.float32)
```
```python
Sizebin = 10000
X4 = cp.zeros(sizebin, dtype=cp.int64)# back to default GPU with index 0 
```

Having establish some basic steps, let’s consider example 1.

**Example 1**: *Write a CuPy program that adds two arrays A and B and store the result in array C. Assume that A and B have 10,000 elements each*.

In [ ]:
import cupy as cp

N = 10000
#select Device with index 1. 
with cp.cuda.Device(1):
    #input data initialzed
    d_A = cp.arange(N, dtype=cp.int32)
    d_B = cp.arange(N, dtype=cp.int32)
    d_C = cp.zeros(N, dtype=cp.int32) # initialize zero filled array
    d_C = d_A + d_B

#optional: copy result from Device to Host 
h_C = cp.asnumpy(d_C)
print(h_C)
#expected output: [    0     2     4 ... 19994 19996 19998]


---
**Exercise 1**: *Follow the steps highlighted above and write a CuPy program to add two arrays. The size of each array is 500,000*. Execute this task in the cell below: 

---


In [ ]:
import cupy as cp








#expected output: [     0      2      4 ... 999994 999996 999998]

## 2-Dimensional Array

In this section, the focus would be on performing simple calculation with 2D arrays. 2D arrays are usually in matrix form and matrix-matrix multiplication operation can be perform on them using CuPy `SGEMM` (Single precision GEneral Matrix Multiplication) and `DGEMM` (Double precision GEneral Matrix Multiplication). Let’s consider two examples of matrix multiplication. First example would be a simple mathematically verifiable `4x4` matrixes `A and B` as shown in figure 2.0, while the second example is a large matrixes `d_A and d_B` of shape `10,000x10,000`. The latter example would use `cuRAND` to randomly generate values for `d_A & d _B` on the Device and python matrix operator `@` based on `cuBLAS` to perform matrix multiplication. 

**Example 2**:  Multiplication of matrix A & B using `cp.dot()` and `@`. 

<center><img src="../images/matrix.png" height="600px" width="506px"></center>
<center><div>Figure 2.0 Matrix A & B multiplication</div></center> 


In [ ]:
N = 4
A = cp.array([[0,0,0,0],[1,1,1,1],[2,2,2,2],[3,3,3,3]],dtype=cp.int32)
B = cp.array([[0,1,2,3],[0,1,2,3],[0,1,2,3],[0,1,2,3]],dtype=cp.int32)

C = cp.dot(A,B)
C2 = A@B
print("dot ops:", C)
print("@ ops:", C2)

#expected output
#dot ops: 
#[[ 0  0  0  0]
# [ 0  4  8 12]
# [ 0  8 16 24]
# [ 0 12 24 36]]
#@ ops: 
#[[ 0  0  0  0]
# [ 0  4  8 12]
# [ 0  8 16 24]
# [ 0 12 24 36]]


**Example 3**:  Multiply matrixes d_A and d_B using Python matrix operator `@`. 

- **Step 1**: initialize matrix size (assume the two matrixes have equal rows and columns)
```python
import cupy as cp
N = 10000
```


- **Step 2**: Fetch or generate matrix values. Matrixes d_A and d_B would be generated using `cuRAND`
```python
d_A = cp.random.random((N,N), dtype=cp.float32)
d_B = cp.random.random(N*N, dtype=cp.float32).reshape(N, N)
```


- **Step 3**: Apply Python matrix operator `@` that uses `cuBLAS`  
```python
d_C = d_A@d_B
print(d_C)
#expected output
...
[2496.929  2493.3096 2512.024  ... 2523.2388 2486.2688 2502.8193]
[2512.366  2522.0713 2518.3489 ... 2529.164  2493.486  2488.1067]
[2493.215  2483.601  2493.606  ... 2523.578  2474.8271 2469.6057]]
```

---
**Exercise 2**: *Write a CuPy program that multiply two matrixes of dimensions 225 x 225. Part of the code has been written for you in the cell below and you are to complete the rest*.

---

In [ ]:
import cupy as cp

N = 225

#generate matrix 



#apply matrix operator @ or cp.dot()



#expected output:
#[[  848610000   848635200   848660400 ...   854204400   854229600 854254800]
# [ 2124360000  2124435825  2124511650 ...  2141193150  2141268975 2141344800]
# [ -894857296  -894730846  -894604396 ...  -866785396  -866658946 -866532496]
# ...
# [  597268464   608532414   619796364 ... -1197101932 -1185837982 -1174574032]
# [ 1873018464  1884333039  1895647614 ...    89886818   101201393 112515968]
# [-1146198832 -1134833632 -1123468432 ...  1376875568  1388240768 1399605968]]

## Kernel Fusion

Kernel fusion is all about fusing functions and it is defined by specifying a decorator `@cp.fuse()` at the top of a user-defined function. Kernel fusion creates and caches the CUDA kernel on it first call in a way that subsequent calls with the same input type are executed on the cached kernel, hence more speed up is gained.

```python
@cp.fuse(kernel_name='<function_name>')
def function_name(<arguments>):
  #<body code> 
```
or as

```python
@cp.fuse()
def function_name(<arguments>):
  #<body code> 
```

**Example 4**: compute  z = $∑_{𝑖=1}$ $𝑥_{𝑖}$ * $𝑤_{𝑖}$

In [ ]:
import cupy as cp

@cp.fuse()
def compute(x,w):
    return cp.sum(x * w)

N = 225
#input data
x = cp.random.random((N), dtype=cp.float32)
w = cp.random.random((N), dtype=cp.float32)

#calling fuse function
z = compute(x,w)
print(z)
#expected output: 57.776024. output may varies because of random values of x & w

## CuPy CUDA Kernels

CuPy CUDA kernels are user defined kernels namely:
- Elementwise Kernels
- Reduction Kernels
- Raw Kernels

### Elementwise Kernels

The elementwise kernel class definition comprises list of input and output arguments with data types specified explicitly or in generic form`(<T>)` that follows C language style. It also includes the kernel body code that denotes computation statement and the kernel name. Note that character `i` and `n`, and variable names that begin with `“_”` are not allowed for use within the elementwise kernel definition. A stepwise example is illustrated below:

**Example 5**: compute r= √($x^2$+$y^2$+$z^2$ ) 

**Step 1**: Set the list of input and output arguments and their data type
```python
input_list = 'float32 d_x, float32 d_y, float32 d_z '
output_list = 'float32 r'
```
you may as well use a generic form of data type as follows:

```python
input_list = 'T d_x, T d_y, T d_z'
output_list = 'T r'
```
**Step 2**: Write the kernel body code to compute the equation
```python
code_body = 'r = sqrt(d_x*d_x + d_y*d_y + d_z*d_z)'
```
**Step 3**: Define elementwise class and set the kernel name
```python
compute_call = cp.ElementwiseKernel(input_list, output_list, code_body,'compute')
```
**Step 4**: Initialize input values
```python
N =2000
d_x = cp.arange(N, dtype=cp.float32)
d_y = cp.arange(N, dtype=cp.float32)
d_z = cp.arange(N, dtype=cp.float32)
r = cp.empty(N, dtype=cp.float32)
```

**step 5**: Make the kernel call
```python
compute_call(d_x,d_y, d_z, r)
print(r)
#expected output: [0.0000000e+00 1.7320508e+00 3.4641016e+00 ... 3.4589055e+03 3.4606375e+03 3.4623696e+03]
```
You can run the above code in the cell below:

In [ ]:
import cupy as cp

input_list = 'float32 d_x, float32 d_y, float32 d_z '
output_list = 'float32 r'
code_body = 'r = sqrt(d_x*d_x + d_y*d_y + d_z*d_z)'

# elementwisekernel class defined
compute_call = cp.ElementwiseKernel(input_list, output_list, code_body,'compute')
# data
N =2000

d_x = cp.arange(N, dtype=cp.float32)
d_y = cp.arange(N, dtype=cp.float32)
d_z = cp.arange(N, dtype=cp.float32)
r = cp.empty(N, dtype=cp.float32)
# kernel call with argument passing
compute_call(d_x,d_y, d_z, r)
print(r)


### Reduction Kernels

Reduction kernels is defined as follows:

    - Input and output arguments with data types specified explicitly or in generic form(<T>) that follows C language style.
    - Identity value that initialized argument to be reduced to zero. 
	- mapping expression that maps each argument values to operands a & b and applies arithmetic operator. 
	- reduction expression that sums operand a & b and stores the output in a, 
	- post mapping expression that executes further operation on operand a. 
	- kernel name

For ease of understanding, `example 6` is used to exemplify reduction kernel.

**Example 6**: Evaluate  z = $∑_{𝑖=1}$ $𝑥_{𝑖}$ * $𝑤_{𝑖}$ + bais


**Step 1**: Set the list of input and output arguments and their data type
```python
input_list = 'float32 x, float32 w, float32 bias'
output_list = 'float32 y'
```
you may as well use a generic form of data type as follows:
```python
input_list = 'T x, T w, T bias'
output_list = 'T y'
```
**Step 2**: set mapping expression
```python
mapping_expr = 'x * w'
```
**Step 3**: set reduction expression `a & b`
```python
reduction_expr= 'a + b'
```
**Step 4**: set post expression for `a`
```python
post_expr = 'y = a + bias'
```
**Step 5**: initialize identity value 0
```python
identity_value = '0'
```
**Step 6**: define reduction kernel class and set the kernel name
```python
dnnLayer = cp.ReductionKernel(
   input_list,
   output_list,
   mapping_expr,
   reduction_expr,
   post_expr,
   identity_value,
   'dnnLayer')
```
**Step 7**: Initialize input values
```python
N = 2000
x = cp.random.random(N, dtype=cp.float32)
w = cp.random.random(N, dtype=cp.float32)
bias = -0.01
```
**Step 8**: make the kernel call
```python
y = dnnLayer(x,w,bias)
print(y)
```
You can run the above code in the cell below:

In [ ]:
import cupy as cp

input_list = 'float32 x, float32 w, float32 bias'
output_list = 'float32 y'
mapping_expr = 'x * w'
reduction_expr= 'a + b'
post_expr = 'y = a + bias'
identity_value = '0'

dnnLayer = cp.ReductionKernel(
   input_list,
   output_list,
   mapping_expr,
   reduction_expr,
   post_expr,
   identity_value,
   'dnnLayer'  )

N = 2000
x = cp.random.random(N, dtype=cp.float32)
w = cp.random.random(N, dtype=cp.float32)
bias = -0.01

y = dnnLayer(x,w,bias)
print(y)


### Raw Kernels

The CuPy Raw kernels are defined through the RawKernel object that enables the direct use of kernels from CUDA source using CUDA’s cuLaunchKernel interface. Raw Kernels are written using CUDA C paradigm therefore there is need to understand the memory architecture to know how best to manipulate threads, thread blocks and grid size. This is important to effectively write Raw Kernels that solve complex task.

### Memory Architecture

When written codes run on the device (GPU), execution is shared amongst threads and blocks of memory space. The execution could be mapped to thousands of threads modelled in blocks and grids form. This idea is illustrated in figure 3.0 with a view that a thread can be seen as a single executing unit on the device.  A `thread block` (also known as a block) is as collection of threads that can communicate, while a collection of these blocks is referred to as a `Grid`. In several devices the maximum number of threads within a thread block is `1,024` and `65,535` blocks within a grid.

<center><img src="../images/thread_blocks.JPG" height="620px" width="540px"/> </center>
 <center><div>Figure 3.0. Thread, block, and grid concept </div></center>


As shown in figure 4.0, the GPU memory space is hierarchically arranged into `shared memory`, `local memory`, `global memory`, `constant memory`, and `texture memory`. Within a block, each thread has its own local memory and register and does communicate with other threads using the shared memory.

<center><img src="../images/memory_architecture.png" height="412px" width="500px"/> </center>
<center><div>Figure 4.0. Memory Architecture</div></center>

**Image source** : <i>Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018</i>.



A raw kernel runs on the Device and it is defined by creating a `RawKernel` object the embeds CUDA C kernel codes. Let’s illustrate this using example 7 as follows:

**Example 7**: Write a CuPy raw kernel program that adds two arrays assume that both arrays contain 10,000 elements each.

**Step 1**:
- First, import `cupy as cp` at the top of your notebook to access `RawKernel` class.
- Next, write an empty raw kernel function enclosed in parenthesis. An example is given below:
```python
import cupy as cp
add_array = cp.RawKernel(r'''
extern "C" __global__
void <function_name>(<arguments>) {

  <body code>
}
''', '<function_name>')
```


- **Write code body**: To successfully write the kernel code body, it is important to know that computations within CUDA kernels execute in thread blocks and grids in a way that input array elements are accessed using global thread id as index. Therefore, it is necessary to uniquely identify distinct threads. A simple illustration on how to estimate global thread `id(s)` is given in figure 5.0 using four blocks of threads stacked over each other to form a matrix in rows and columns arrangement. Global thread ids are calculated in `x-dimension` (ideally thread block are in x,y,z dimensions) by rearranging the thread blocks as single row and then estimate using statement below:

```python
tid = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
```
<center><img src="../images/thread_position.png" height="350px" width="350px"/></center>
<center><div>Figure 5.0 Estimating thread id for threads in green and orange</div></center>

Now that we know how to compute global thread ids, we can proceed to write the CUDA C body code within the Raw Kernel as follows:

```python
import cupy as cp
N = 10000 #initialize array size
add_array = cp.RawKernel(r'''
extern "C" __global__
void addFunc(const int* d_A, const int* d_B, int* d_C ) {
 int tid = blockDim.x * blockIdx.x + threadIdx.x;
 d_C[tid]= d_A[tid] + d_B[**tid];
}
''', 'addFunc')
```

**Step 2**:

- **Write the Host code**: The first thing to do is to initialize your input arrays as follows
```python
import numpy as np
h_A = np.arange(N, dtype=np.int32)
h_B = np.arange(N, dtype=np.int32)
```
Do data transfer by copying data (input array) from the `Host` to the `Device` using `cp.asarray()` function.

```python
d_A = cp.asarray(h_A)
d_B = cp.asarray(h_B)
d_C = cp.zeros(N, dtype=cp.int32) # initialize zero filled array
```

**Step 3**:

The next step is to call the raw kernel function from the Host. But before that, a vital move would be to initialize the number of threads that would make up a single block (thread block) so that number of blocks required in a grid to execute the raw kernel can be estimated. In CuPy, raw kernel calls have a definition pattern as follows:
```python
<raw_kernel_name>((<num_of_blocks_per_grid>),(<num_of_threads_per_block>),(<arguments>))
```
The total number of threads required is equivalent to the size of initialized array, which is 10,000, therefore:

```python
num_of_threads_per_block = 256 # this has not exceeded the limit i.e < 1024
```
Then, `num_of_blocks_per_grid` can be estimated as:
```python
num_of_blocks_per_grid = math.ceil (N / num_of_threads_per_block)
```
Subsequently, the raw kernel function is called this way:

```python
add_array((num_of_blocks_per_grid,),(num_of_threads_per_block,),(d_A, d_B, d_C))
```

**Step 4**:

Copy result from Device to Host using `cp.asnumpy()` function, thus:

```python
h_C = cp.asnumpy(d_C)
```

You can run the entire code in the cell below.

In [ ]:
import cupy as cp
import numpy as np
import math

N = 10000 #initialize array size
add_array = cp.RawKernel(r'''
extern "C" __global__
void addFunc(const int* d_A, const int* d_B, int* d_C ) {
 int tid = blockDim.x * blockIdx.x + threadIdx.x;
 d_C[tid]= d_A[tid] + d_B[tid];

}
''', 'addFunc')

h_A = np.arange(N, dtype=np.int32)
h_B = np.arange(N, dtype=np.int32)

d_A = cp.asarray(h_A)
d_B = cp.asarray(h_B)
d_C = cp.zeros(N, dtype=cp.int32) # initialize zero filled array

num_of_threads_per_block = 256
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)

add_array((num_of_blocks_per_grid,),(num_of_threads_per_block,),(d_A, d_B, d_C))
h_C = cp.asnumpy(d_C)
print(h_C)

#expected output: [    0     2     4 ... 19994 19996 19998]

---
**Exercise 3**: *Follow the steps highlighted above and write a CuPy Raw Kernel program that multiply two arrays and store the result in a third array. The size of each array is `500,000`. Execute this task in the cell below:*

---

In [ ]:
import cupy as cp
import numpy as np
import math

N = 500000 #initialize array size



#expected output[ 0 1 4 ... 888896841 889896836 890896833]

### Raw Modules

The Raw Modules has the same procedure as the Raw Kernel. In addition to this, several CUDA C kernel functions can be included within in form of a module as the name connote. Each kernel function within the Raw Module can be accessed by instantiating the  object of `RawModule` class and a call to `get_function()` method. 

**Example 8:** 
(i) z = $∑_{𝑖=1}$ $𝑥_{𝑖}$ * $𝑤_{𝑖}$      

(ii) r= √($x^2$+$y^2$+$z^2$)

The two tasks in `example 8` are solved using raw module approach. Kernel `sum_mul` and `compute_xyz` proffer solutions to example 8(i) and 8(ii) respectively. In the `sum_mul` kernel, `__syncthread()` was used to synchronize threads in blocks in a way that all threads within a block complete the multiplication operation before moving ahead to the sum operation. The `atomicAdd()` method helps avoid incorrect sum by preventing multiple threads from performing addition operation at same time, thus, only a single thread is allowed at a time. Note that this is not the best approach, it is however written this way to reduce complexity at this level.  

```python
raw_module_code = r'''
extern "C" {
            __global__ void sum_mul(float* d_x, float* d_w, float* d_z) 
            {
                 float sum[2000];
                 int tid = blockDim.x * blockIdx.x + threadIdx.x;
                 sum[tid] = d_x[tid] * d_w[tid];
                 __syncthreads();
                 atomicAdd(d_z, sum[tid]);
            }
            __global__ void compute_xyz(float* x, float* y, float* z, float* r ) 
            {
                int tid = blockDim.x * blockIdx.x + threadIdx.x;
                r[tid] = sqrt(x[tid] * x[tid] + y[tid] * y[tid] + z[tid] * z[tid]) ;

            }
       }
'''
```
The next step is to load the raw module by creating an object.
```python
#loading module through RawModule object
raw_module_object = cp.RawModule(code = raw_module_code)
```
Get the kernels within the raw module through the `get_function()` method
```python
#acessing kernels within the Raw module
sum_mul = raw_module_object.get_function('sum_mul')
compute_xyz = raw_module_object.get_function('compute_xyz')
```
Initialize data size, thread block size and, grid size
```python
#data
N = 2000 #initialize array size
num_of_threads_per_block = 128
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)
```
Initialize data for example 8(i) and copy data to the Device using `cp.asarray()`
```python
h_x = np.arange(N, dtype=np.float32)
h_w = np.arange(N, dtype=np.float32)

d_x = cp.asarray(h_x)
d_w = cp.asarray(h_w)
d_z = cp.zeros(1, dtype=cp.float32)# initialize zero
```

Call kernel `sum_mul` and pass the required arguments
```python
sum_mul((num_of_blocks_per_grid,),(num_of_threads_per_block,),(d_x, d_w, d_z))
h_z = cp.asnumpy(d_z)
print("h_z:", h_z)

verifying result
print("non kernel:", cp.sum(h_x * h_w))
```
Initialize data for example 8(ii) directly on the Device using `cp.arange()`

```python
x = cp.arange(N, dtype=cp.float32)
y = cp.arange(N, dtype=cp.float32)
z = cp.arange(N, dtype=cp.float32)
r = cp.empty(N, dtype=cp.float32)
```
Call kernel `compute_xyz` and pass the required arguments 
```python
compute_xyz((num_of_blocks_per_grid,),(num_of_threads_per_block,),(x, y, z, r))
h_r = cp.asnumpy(r)
print("h_r:", h_r)

Verifying result
print("non kernel:", cp.sqrt(x * x + y * y+ z * z ))
#expected result: 
h_z: [2.6646702e+09]
h_r: [0.0000000e+00 1.7320508e+00 3.4641016e+00 ... 3.4589055e+03 3.4606375e+03 3.4623696e+03]
```


## JIT Kernel

The JIT kernel is defined through the `cupyx.jit.rawkernel` decorator. It uses the same concept as the raw kernel but differs by using python functions rather than CUDA C kernels. The decorator the specify at the top of a python function, hence the function becomes a JIT kernel. Let’s illustrate this using `example 7` from the raw kernel section.

Firstly, `import jit from cupyx` library

```python
import cupy as cp
from cupyx import jit
```
Next, write the Jit kernel

```python
@jit.rawkernel()
def addFunc(d_A, d_B, d_C):
    tid = jit.blockDim.x * jit.blockIdx.x + jit.threadIdx.x
    d_C[tid] = d_A[tid] + d_B[tid]
```
Initialize data size, thread block size and, grid size

```python
N = 10000 #initialize array size
num_of_threads_per_block = 128
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)
```
Initialize data directly on the Device using `cp.arange()`

```python
d_A = cp.arange(N, dtype=cp.float32)
d_B = cp.arange(N, dtype=cp.float32)
d_C = cp.zeros(N, dtype=cp.int32) # initialize zero filled array
```
Call jit kernel `addFunc` and pass the required arguments
```python
addFunc((num_of_blocks_per_grid,),(num_of_threads_per_block,),(d_A, d_B, d_C))

print("d_C:", d_C)

#expected output: d_C: [    0     2     4 ... 19994 19996 19998]
```

There are two notable actions in the code above, first, data was not initialized on the Host but directly on the Device; second, the output of the `d_C` was not copy to the Host using `asnumpy()` but was used directly on the Host. This shows data visibility, however, it is not the best data management approach in some context. Please run the cell below: 

In [ ]:
import cupy as cp
from cupyx import jit
import math

@jit.rawkernel()
def addFunc(d_A, d_B, d_C):
    tid = jit.blockDim.x * jit.blockIdx.x + jit.threadIdx.x
    d_C[tid] = d_A[tid] + d_B[tid]

N = 100000 #initialize array size
num_of_threads_per_block = 128
num_of_blocks_per_grid = math.ceil(N / num_of_threads_per_block)

d_A = cp.arange(N, dtype=cp.int32)
d_B = cp.arange(N, dtype=cp.int32)
d_C = cp.zeros(N, dtype=cp.int32) # initialize zero filled array
addFunc((num_of_blocks_per_grid,),(num_of_threads_per_block,),(d_A, d_B, d_C))
print("d_C:", d_C)

#expected output: [    0     2     4 ... 19994 19996 19998]

---
**Exercise 4**: *Follow the steps highlighted above and write a CuPy Raw Kernel program that multiply two arrays and store the result in a third array. The size of each array is 500,000. Execute this task in the cell below:*

---

In [ ]:
import cupy as cp
import numpy as np
import math

N = 500000 #initialize array size







#expected output: [  0   1  4 ... 2147483647 2147483647 2147483647]

## Summary
<img src="../images/cupy_summary.png" width="80%" height="80%">

---
## Lab Task

In this section, you are expected to click on the **Serial Code Lab Assignment** link and proceed to Lab 2. In this lab, you will find three python serial code functions. You are required to revise the **pair_gpu** function to run on the GPU, and likewise do a few modifications within the **main** function.

## <center><div style="text-align:center; color:#FF0000; border:3px solid red;height:80px;"> <b><br/> [Serial Code Lab Assignment](serial_RDF.ipynb) </b> </div></center>

 
---


## Post-Lab Summary

If you would like to download this lab for later viewing, we recommend you go to your browser's File menu (not the Jupyter notebook file menu) and save the complete web page. This will ensure the images are copied as well. You can also execute the following cell block to create a zip-file of the files you've been working on and download it with the link below.

In [ ]:
%%bash
cd ..
rm -f nways_files.zip
zip -r nways_files.zip *


**After** executing the above zip command, you should be able to download and save the zip file by holding down <mark>Shift</mark> and <mark>Right-Clicking</mark> [Here](../nways_files.zip).

**IMPORTANT**: Please click on **HOME** to go back to the main notebook for *N ways of GPU programming for MD* code.

---
# <center><div style="text-align: center;border:3px; border-style:solid; border-color:#FF0000; padding: 1em;"> [HOME](../../../nways_MD_start_python.ipynb)</div></center>

---

# Links and Resources

[NVIDIA Nsight System](https://docs.nvidia.com/nsight-systems/)

[NVIDIA CUDA Toolkit](https://developer.nvidia.com/cuda-downloads)

**NOTE**: To be able to see the Nsight System profiler output, please download Nsight System latest version from [here](https://developer.nvidia.com/nsight-systems).

Don't forget to check out additional [OpenACC Resources](https://www.openacc.org/resources) and join our [OpenACC Slack Channel](https://www.openacc.org/community#slack) to share your experience and get more help from the community.

---


##  References
- https://docs.cupy.dev/en/stable/
- https://cupy.dev/
- CuPy Documentation Release 8.5.0, Preferred Networks, inc. and Preferred Infrastructure inc., Feb 26, 2021.
- Bhaumik Vaidya, Hands-On GPU-Accelerated Computer Vision with OpenCV and CUDA, Packt Publishing, 2018.
- Crissman Loomis and Emilio Castillo, CuPy Overview: NumPy Syntax Computation with Advanced CUDA Features, GTC Digital March, March 2020.
- https://www.gpuhackathons.org/technical-resources
- https://rapids.ai/start.html

--- 

## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).